In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
"""import zipfile

# Path to the Akhbarona zip file
akhbarona_zip_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets/Akhbarona.zip'

# Extract the Akhbarona dataset
with zipfile.ZipFile(akhbarona_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Arabic_Sentiment_Tweets/Akhbarona')


SyntaxError: incomplete input (<ipython-input-2-b1aa9b147e07>, line 1)

In [ ]:
"""# Path to the Arabiya zip file
arabiya_zip_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets/Arabiya.zip'

# Extract the Arabiya dataset
with zipfile.ZipFile(arabiya_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Arabic_Sentiment_Tweets/Arabiya')


In [ ]:
"""# Path to the Khaleej zip file
khaleej_zip_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets/Khaleej.zip'

# Extract the Khaleej dataset
with zipfile.ZipFile(khaleej_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Arabic_Sentiment_Tweets/Khaleej')


In [ ]:
"""# Path to the SANAD_SUBSET zip file
sanad_subset_zip_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets/SANAD_SUBSET.zip'

# Extract the SANAD_SUBSET dataset
with zipfile.ZipFile(sanad_subset_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Arabic_Sentiment_Tweets/SANAD_SUBSET')


In [ ]:
"""import os

# Path to the dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets'

# List all files in the directory
dataset_files = os.listdir(dataset_path)

# Print the list of files
for file in dataset_files:
    print(file)


In [ ]:
"""# List the contents of each folder
for folder in dataset_files:
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):  # Check if it's a directory
        print(f"Files in {folder}:")
        files_in_folder = os.listdir(folder_path)
        for file in files_in_folder:
            print(file)
    else:
        print(f"{folder} is a file, not a directory.")


In [ ]:
"""# Path to a sample text file (e.g., one of the .txt files)
sample_file_path = '/content/drive/MyDrive/Arabic_Sentiment_Tweets/Akhbarona/Culture/5002.txt'

# Open the file and read its contents
with open(sample_file_path, 'r', encoding='utf-8') as file:
    sample_data = file.read()

# Print the first 500 characters to inspect the content
print(sample_data[:500])


In [3]:
!pip install transformers torch torchmetrics


In [4]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch

# Load tokenizer
model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load your dataset here
texts = ["Arabic text example 1", "Arabic text example 2"]  # Replace with actual dataset
labels = [1, 0]  # Example labels

# Tokenize and split
encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
train_texts, val_texts, train_labels, val_labels = train_test_split(
    encodings['input_ids'], labels, test_size=0.2, random_state=42
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics import Accuracy
import torch.nn as nn

# Prepare data
train_dataset = TensorDataset(train_texts, torch.tensor(train_labels))
val_dataset = TensorDataset(val_texts, torch.tensor(val_labels))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Training setup
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Define the accuracy metric for a binary classification task
metric = Accuracy(task="binary").to(device)

# Training loop
for epoch in range(50):
    model.train()
    for batch in train_loader:
        input_ids, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 1.2349765300750732
Epoch 2, Loss: 0.6395505666732788
Epoch 3, Loss: 0.5692559480667114
Epoch 4, Loss: 0.2551383972167969
Epoch 5, Loss: 0.16202670335769653
Epoch 6, Loss: 0.11991899460554123
Epoch 7, Loss: 0.1319114714860916
Epoch 8, Loss: 0.08415491878986359
Epoch 9, Loss: 0.06294127553701401
Epoch 10, Loss: 0.10745317488908768
Epoch 11, Loss: 0.08482898771762848
Epoch 12, Loss: 0.07753069698810577
Epoch 13, Loss: 0.09774982184171677
Epoch 14, Loss: 0.10793442279100418
Epoch 15, Loss: 0.0872984305024147
Epoch 16, Loss: 0.040963053703308105
Epoch 17, Loss: 0.045471735298633575
Epoch 18, Loss: 0.02772335149347782
Epoch 19, Loss: 0.04151534289121628
Epoch 20, Loss: 0.030503900721669197
Epoch 21, Loss: 0.02515443228185177
Epoch 22, Loss: 0.020524965599179268
Epoch 23, Loss: 0.030639639124274254
Epoch 24, Loss: 0.02565556950867176
Epoch 25, Loss: 0.01803230307996273
Epoch 26, Loss: 0.02105235680937767
Epoch 27, Loss: 0.010230128653347492
Epoch 28, Loss: 0.014894470572471619


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadDiffAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadDiffAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads

        # Linear transformations for query, key, and value
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

        # Output layer
        self.dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """Split the input tensor into multiple heads."""
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)  # Shape: (batch_size, num_heads, seq_len, depth)

    def attention(self, query, key, value, mask=None):
        """Compute the scaled dot-product attention."""
        matmul_qk = torch.matmul(query, key.transpose(-2, -1))  # (batch_size, num_heads, seq_len, seq_len)

        # Scale matmul_qk
        dk = query.size()[-1]
        scaled_attention_logits = matmul_qk / torch.sqrt(torch.tensor(dk, dtype=torch.float32))

        # Apply mask (if any)
        if mask is not None:
            # Expand the mask to match the dimensions of scaled_attention_logits
            mask = mask[:, None, None, :].repeat(1, self.num_heads, scaled_attention_logits.size(2), 1)
            scaled_attention_logits += (mask * -1e9)  # Large negative number for padding positions

        attention_weights = F.softmax(scaled_attention_logits, dim=-1)  # Shape: (batch_size, num_heads, seq_len, seq_len)

        # Compute the attention output
        output = torch.matmul(attention_weights, value)  # Shape: (batch_size, num_heads, seq_len, depth)

        return output, attention_weights

    def forward(self, input_ids, attention_mask=None):
        batch_size = input_ids.size(0)

        # Compute query, key, value projections
        query = self.split_heads(self.query(input_ids), batch_size)
        key = self.split_heads(self.key(input_ids), batch_size)
        value = self.split_heads(self.value(input_ids), batch_size)

        # Compute attention
        output, attention_weights = self.attention(query, key, value, mask=attention_mask)

        # Concatenate heads and pass through the output layer
        output = output.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)  # Shape: (batch_size, seq_len, d_model)
        output = self.dense(output)

        return output, attention_weights


In [7]:
from transformers import BertForSequenceClassification

class CustomBERTWithAttention(nn.Module):
    def __init__(self, original_model, attention_layer):
        super(CustomBERTWithAttention, self).__init__()
        self.bert = original_model.bert  # BERT base model
        self.attention = attention_layer  # Custom attention layer
        self.classifier = original_model.classifier  # Classifier for downstream tasks

    def forward(self, input_ids, attention_mask=None):
        # Get BERT outputs
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs[0]  # Shape: (batch_size, seq_len, hidden_size)

        # Apply custom attention
        attention_output, attention_weights = self.attention(sequence_output, attention_mask)

        # Apply the classifier to the output of the attention layer
        logits = self.classifier(attention_output[:, 0, :])  # Use [CLS] token's output for classification
        return logits


In [10]:
import os
import random
from sklearn.model_selection import train_test_split

# Define the paths to your folders
folders = ['Akhbarona', 'Arabiya', 'Khaleej', 'SANAD_SUBSET']

# Initialize empty lists for texts and labels
texts = []
labels = []

# Number of files to sample from each folder
sample_size = 20  # Adjust this number based on how small you want the sample to be

# Loop through each folder and load a small sample of text files
for folder in folders:
    folder_path = os.path.join('/content/drive/MyDrive/Arabic_Sentiment_Tweets/', folder)  # Replace with your actual data path
    for subfile in os.listdir(folder_path):
        subfile_path = os.path.join(folder_path, subfile)
        if os.path.isdir(subfile_path):  # Check if it is a subfolder
            text_files = [f for f in os.listdir(subfile_path) if f.endswith('.txt')]
            sampled_files = random.sample(text_files, min(len(text_files), sample_size))  # Sample up to `sample_size` files
            for text_file in sampled_files:
                text_file_path = os.path.join(subfile_path, text_file)
                with open(text_file_path, 'r', encoding='utf-8') as file:
                    text = file.read()  # Read the content of the text file
                    texts.append(text)
                    labels.append(folder)  # Use the folder name as the label

# Check the lengths of your lists
print(len(texts), len(labels))

# Split the dataset into train and validation sets (using 20% for validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# Check the split
print(len(train_texts), len(val_texts), len(train_labels), len(val_labels))


200 200
160 40 160 40


In [11]:
from transformers import BertTokenizer
import torch

# Initialize the tokenizer (assuming you're using BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts for both training and validation sets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Convert labels to tensor format
label_mapping = {'Akhbarona': 0, 'Arabiya': 1, 'Khaleej': 2, 'SANAD_SUBSET': 3}
train_labels = torch.tensor([label_mapping[label] for label in train_labels])
val_labels = torch.tensor([label_mapping[label] for label in val_labels])

# Create PyTorch dataset
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets for DataLoader
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

# Create DataLoaders for training and validation
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)


In [14]:
from transformers import BertForSequenceClassification, AdamW

# Initialize model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)  # 4 classes in your dataset
model.to(device)

# Set optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(30):  # Train for 3 epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()  # Reset gradients from previous step
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1} complete!')

# Save the model after training
model.save_pretrained('./fine_tuned_model')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 complete!
Epoch 2 complete!
Epoch 3 complete!
Epoch 4 complete!
Epoch 5 complete!
Epoch 6 complete!
Epoch 7 complete!
Epoch 8 complete!
Epoch 9 complete!
Epoch 10 complete!
Epoch 11 complete!
Epoch 12 complete!
Epoch 13 complete!
Epoch 14 complete!
Epoch 15 complete!
Epoch 16 complete!
Epoch 17 complete!
Epoch 18 complete!
Epoch 19 complete!
Epoch 20 complete!
Epoch 21 complete!
Epoch 22 complete!
Epoch 23 complete!
Epoch 24 complete!
Epoch 25 complete!
Epoch 26 complete!
Epoch 27 complete!
Epoch 28 complete!
Epoch 29 complete!
Epoch 30 complete!


In [15]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        # Get predictions
        predictions = torch.argmax(logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6750
